# Tabela aerodromos 

 - Descrever qual estratégia você usaria para ingerir estes dados de forma incremental caso precise capturar esses dados a cada mes?
 
 R: Pela api necessitar acesso a plataforma para liberar o acesso, penso em desevolver uma automação com selenium e salvar como .exe. Com o selenium posso crar passos para a máquina executar, desde acessar a pagina, logar na minha conta, abrir o código de ingestão, tratamento dos mesmo e armazenamento em uma data lake especifico, assim posso colocar no agendador de tarefas e definir a data e hora para o script ser executado.
 
 

## Desafios abordados
- Criar nova tabela aerodromos
  - Através da API [https://rapidapi.com/Active-api/api/airport-info/]() trazer os aeródramos através do código ICAO presente nos dados de VRA.
  - Salvar estes dados
  

## Seleção dos ICAO dos aerodromos
    Para fazer a requisição da api pelo ICAO do aerodromos selecionei as colunas icaoaeródromo_origem e icaoaeródromo_destino para ter os ICAO dos aerodromos. 
    


In [77]:
# Bibliotecas usadas
import numpy as np
import pandas as pd
import os
import ast
import json
import re

import http.client
import time

In [80]:
def snake_case(valor):
    valor = re.findall('[A-Z]*[^A-Z]*', valor) # dá um split nas letras maiúsculas
    if valor[0] == '_':
        valor[0] = ''
    valor = [x.lower() for x in valor]
    return '_'.join(valor)[:-1]

In [103]:
# Carregando os dados .json
caminho_arquivo = 'C:/Users/Didrich/Documents/Python Scripts/DataEnginnerTest/VRA/VRA_Brutos/VRA_20218.json'  

if not os.path.isfile(caminho_arquivo):
    exit(1)  

with open(caminho_arquivo, 'r', encoding='utf-8-sig') as arquivo:
    conteudo_arquivo = arquivo.read()
    df = json.loads(conteudo_arquivo)
    df_vra = pd.DataFrame(df)
    
# Aplicar a função a todas as colunas do DataFrame
df_vra.columns = [snake_case(col) for col in df_vra.columns]


# Verificar o resultado
df_vra.head()

,icaoempresa_aérea,número_voo,código_autorização,código_tipo_linha,icaoaeródromo_origem,icaoaeródromo_destino,partida_prevista,partida_real,chegada_prevista,chegada_real,situação_voo,código_justificativa
0,IPM,5555,0,N,SBGL,SBGR,2021-08-15 19:15:00,None,2021-08-15 20:10:00,None,NÃO INFORMADO,N/A
1,IPM,5555,0,N,SBGL,SBGR,2021-08-16 19:25:00,None,2021-08-16 20:25:00,None,NÃO INFORMADO,N/A
2,AEA,0057,0,X,LEMD,SBGR,2021-08-18 18:45:00,2021-08-18 19:19:00,2021-08-19 05:25:00,2021-08-19 04:51:00,REALIZADO,N/A
3,AEA,0057,0,X,LEMD,SBGR,2021-08-08 18:45:00,2021-08-08 19:08:00,2021-08-09 05:25:00,2021-08-09 05:13:00,REALIZADO,N/A
4,AEA,0057,0,X,LEMD,SBGR,2021-08-29 18:45:00,2021-08-29 18:54:00,2021-08-30 05:25:00,2021-08-30 04:59:00,REALIZADO,N/A


In [45]:
# Selecionando as icao para fazer a requisão da api
lista1 = df['icaoaeródromo_origem'].unique()
lista2 = df['icaoaeródromo_destino'].unique()

# Unir as duas listas e obter valores únicos usando numpy.unique
lista_icaos = np.unique(np.concatenate((lista1, lista2)))

# Exibir a lista única
print(lista_icaos)


['CYYZ' 'DGAA' 'DNKN' 'DNMM' 'EBBR' 'EBOS' 'EDDF' 'EGLL' 'EHAM' 'EHBK'
 'ELLX' 'FNLU' 'GCTS' 'GOBD' 'GUCY' 'GVAC' 'HAAB' 'KAEX' 'KATL' 'KDFW'
 'KEWR' 'KFLL' 'KHSV' 'KIAH' 'KIWA' 'KJFK' 'KLAX' 'KMEM' 'KMIA' 'KORD'
 'KPVG' 'KSTL' 'LEMD' 'LFPG' 'LIMC' 'LIRF' 'LPPR' 'LPPT' 'LSZH' 'LTBA'
 'LTFM' 'MDPC' 'MGGT' 'MMMX' 'MMUN' 'MPTO' 'MTPP' 'MUHA' 'OEJN' 'OMDB'
 'OTHH' 'PANC' 'RKSI' 'SAAR' 'SABE' 'SAEZ' 'SBAC' 'SBAE' 'SBAR' 'SBAT'
 'SBAU' 'SBBE' 'SBBG' 'SBBR' 'SBBV' 'SBBW' 'SBCA' 'SBCB' 'SBCF' 'SBCG'
 'SBCH' 'SBCJ' 'SBCN' 'SBCP' 'SBCR' 'SBCT' 'SBCX' 'SBCY' 'SBCZ' 'SBDB'
 'SBDN' 'SBEG' 'SBFI' 'SBFL' 'SBFN' 'SBFZ' 'SBGL' 'SBGO' 'SBGR' 'SBGV'
 'SBHT' 'SBIH' 'SBIL' 'SBIZ' 'SBJA' 'SBJD' 'SBJE' 'SBJI' 'SBJP' 'SBJU'
 'SBJV' 'SBKG' 'SBKP' 'SBLO' 'SBMA' 'SBMD' 'SBME' 'SBMG' 'SBMK' 'SBML'
 'SBMO' 'SBMQ' 'SBMS' 'SBNF' 'SBNM' 'SBPA' 'SBPB' 'SBPJ' 'SBPK' 'SBPL'
 'SBPS' 'SBPV' 'SBRB' 'SBRD' 'SBRF' 'SBRJ' 'SBRP' 'SBSG' 'SBSI' 'SBSJ'
 'SBSL' 'SBSM' 'SBSN' 'SBSP' 'SBSR' 'SBSV' 'SBTB' 'SBTC' 'SBTE' 'SBTF'
 'SBTG

## Requisição da API
    Com a lista das ICAO unicas dos aerodromos de origem e destino, podemos fazer a requisição para a API através da minha conta na conta rapidapi. A requisição foi feita com um laço de repetição para trazer os dados de cada ICAO na lista

In [85]:
import http.client
import pandas as pd
import json

conn = http.client.HTTPSConnection("airport-info.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "3fa013dc2amshaaf9de32033c66fp14d2e8jsn3c5af2980b7c",
    'X-RapidAPI-Host': "airport-info.p.rapidapi.com"
}

# Lista para armazenar os dados
dados_icaos = []

# Iterar sobre a lista de códigos ICAO e fazer solicitações para cada um
for icao in lista_icaos:
    endpoint = f"/airport?icao={icao}"
    conn.request("GET", endpoint, headers=headers)
    res = conn.getresponse()
    data = res.read()
    
    print(f"Dados para o ICAO {icao}:")
    print(data.decode("utf-8"))
    
    # Converter os dados JSON para um dicionário Python
    dados_json = json.loads(data.decode("utf-8"))
    
    # Adicione os dados ao lista
    dados_icaos.append(dados_json)

# Converter a lista de dicionários em um DataFrame pandas
df = pd.DataFrame(dados_icaos)

Dados para o ICAO CYYZ:
{"id":8847,"iata":"YYZ","icao":"CYYZ","name":"Toronto Pearson International Airport","location":"Toronto, Ontario, Canada","street_number":"6301","street":"Silver Dart Drive","city":"Mississauga","county":"","state":"Ontario","country_iso":"CA","country":"Canada","postal_code":"L5P 1B2","phone":"+1 416-247-7678","latitude":43.67772,"longitude":-79.62482,"uct":-240,"website":"http://www.torontopearson.com/"}

Dados para o ICAO DGAA:
{"id":53,"iata":"ACC","icao":"DGAA","name":"Kotoka International Airport","location":"Accra, Ghana","street_number":"","street":"Airport Road","city":"Accra","county":"Accra Metropolis","state":"Greater Accra Region","country_iso":"GH","country":"Ghana","postal_code":"","phone":"+233 30 277 6171","latitude":5.6060953,"longitude":-0.1681235,"uct":0,"website":"http://www.gacl.com.gh/"}

Dados para o ICAO DNKN:
{"id":3531,"iata":"KAN","icao":"DNKN","name":"Mallam Aminu Kano International Airport","location":"Kano, Nigeria","street_number

Dados para o ICAO KEWR:
{"id":2214,"iata":"EWR","icao":"KEWR","name":"Newark Liberty International Airport","location":"Newark, New Jersey, United States","street_number":"3","street":"Brewster Road","city":"Newark","county":"","state":"New Jersey","country_iso":"US","country":"United States","postal_code":"7114","phone":"+1 973-961-6000","latitude":40.689533,"longitude":-74.17446,"uct":-240,"website":"http://www.panynj.gov/airports/newark-liberty.html"}

Dados para o ICAO KFLL:
{"id":2318,"iata":"FLL","icao":"KFLL","name":"Fort Lauderdale–Hollywood International Airport","location":"Fort Lauderdale / Hollywood, Florida, United States","street_number":"100","street":"Terminal Drive","city":"Fort Lauderdale","county":"Broward County","state":"Florida","country_iso":"US","country":"United States","postal_code":"33315","phone":"+1 954-359-1200","latitude":26.074234,"longitude":-80.150604,"uct":-240,"website":"http://www.fll.net/"}

Dados para o ICAO KHSV:
{"id":3020,"iata":"HSV","icao":"K

Dados para o ICAO LTFM:
{"id":3290,"iata":"ISL","icao":"LTFM","name":"Istanbul New Airport","location":"Istanbul, Turkey","street_number":"","street":"","city":"","county":"Bakırköy","state":"Istanbul","country_iso":"TR","country":"Turkey","postal_code":"34149","phone":"+90 212 463 30 00","latitude":40.98299,"longitude":28.810442,"uct":180,"website":"http://www.ataturkairport.com/"}

Dados para o ICAO MDPC:
{"id":6160,"iata":"PUJ","icao":"MDPC","name":"Punta Cana International Airport","location":"Punta Cana, Dominican Republic","street_number":"","street":"Carretera Aeropuerto","city":"Punta Cana","county":"","state":"La Altagracia","country_iso":"DO","country":"Dominican Republic","postal_code":"23000","phone":"+1 809-959-2376","latitude":18.562244,"longitude":-68.368355,"uct":-240,"website":"http://www.puntacanainternationalairport.com/"}

Dados para o ICAO MGGT:
{"id":2732,"iata":"GUA","icao":"MGGT","name":"La Aurora International Airport","location":"Guatemala City, Guatemala","st

Dados para o ICAO SBBE:
{"id":633,"iata":"BEL","icao":"SBBE","name":"Val de Cans International Airport","location":"Belém, Pará, Brazil","street_number":"s/n","street":"Avenida Júlio César","city":"","county":"Belém","state":"Pará","country_iso":"BR","country":"Brazil","postal_code":"66115-970","phone":"+55 91 3210-6000","latitude":-1.3820615,"longitude":-48.477524,"uct":-180,"website":"http://www4.infraero.gov.br/aeroportos/aeroporto-internacional-de-belem-val-de-cans-julio-cezar-ribeiro/"}

Dados para o ICAO SBBG:
{"id":693,"iata":"BGX","icao":"SBBG","name":"Comandante Gustavo Kraemer Airport","location":"Bagé, Rio Grande do Sul, Brazil","street_number":"","street":"","city":"Bagé","county":"Bagé","state":"Rio Grande do Sul","country_iso":"BR","country":"Brazil","postal_code":"","phone":"+55 53 3242-7702","latitude":-31.389591,"longitude":-54.11207,"uct":-180,"website":"http://www4.infraero.gov.br/aeroportos/aeroporto-internacional-de-bage-comandante-gustavo-kraemer/"}

Dados para o 

Dados para o ICAO SBDN:
{"id":6054,"iata":"PPB","icao":"SBDN","name":"Presidente Prudente Airport","location":"Presidente Prudente, São Paulo, Brazil","street_number":"s/nº","street":"Assis Chateaubriand, km 459","city":"","county":"Presidente Prudente","state":"São Paulo","country_iso":"BR","country":"Brazil","postal_code":"19053-680","phone":"+55 18 3222-8099","latitude":-22.17834,"longitude":-51.417664,"uct":-180,"website":"http://www.daesp.sp.gov.br/aeroporto-detalhe/?id=879"}

Dados para o ICAO SBEG:
{"id":4500,"iata":"MAO","icao":"SBEG","name":"Eduardo Gomes International Airport","location":"Manaus, Amazonas, Brazil","street_number":"1350","street":"Av. Santos Dumont","city":"","county":"Manaus","state":"Amazonas","country_iso":"BR","country":"Brazil","postal_code":"69041-000","phone":"+55 92 3652-1210","latitude":-3.0358474,"longitude":-60.046326,"uct":-240,"website":"http://www4.infraero.gov.br/aeroportos/aeroporto-internacional-de-manaus-eduardo-gomes/"}

Dados para o ICAO SB

Dados para o ICAO SBKG:
{"id":1491,"iata":"CPV","icao":"SBKG","name":"Campina Grande Airport (Presidente João Suassuna Airport)","location":"Campina Grande, Paraíba, Brazil","street_number":"S/N","street":"Avenida Uberaba","city":"","county":"Campina Grande","state":"Paraíba","country_iso":"BR","country":"Brazil","postal_code":"58418-410","phone":"+55 83 3332-9023","latitude":-7.2688785,"longitude":-35.89402,"uct":-180,"website":"http://www4.infraero.gov.br/aeroportos/aeroporto-de-campina-grande-presidente-joao-suassuna"}

Dados para o ICAO SBKP:
{"id":7941,"iata":"VCP","icao":"SBKP","name":"Viracopos/Campinas International Airport","location":"São Paulo / Campinas, São Paulo, Brazil","street_number":"km 66","street":"Rodovia Santos Dumont","city":"","county":"Campinas","state":"São Paulo","country_iso":"BR","country":"Brazil","postal_code":"13055-900","phone":"+55 19 3725-5000","latitude":-23.008205,"longitude":-47.13757,"uct":-180,"website":"http://www.viracopos.com/"}

Dados para o 

Dados para o ICAO SBPS:
{"id":908,"iata":"BPS","icao":"SBPS","name":"Porto Seguro Airport","location":"Porto Seguro, Bahia, Brazil","street_number":"68","street":"Rua Primeiro de Maio","city":"","county":"Porto Seguro","state":"Bahia","country_iso":"BR","country":"Brazil","postal_code":"45810-000","phone":"+55 73 3288-1880","latitude":-16.439308,"longitude":-39.081562,"uct":-180,"website":"http://www.aeroportoseguro.com.br/"}

Dados para o ICAO SBPV:
{"id":6182,"iata":"PVH","icao":"SBPV","name":"Governador Jorge Teixeira de Oliveira International Airport","location":"Porto Velho, Rondônia, Brazil","street_number":"S/N","street":"Avenida Governador Jorge Teixeira","city":"","county":"Porto Velho","state":"Rondônia","country_iso":"BR","country":"Brazil","postal_code":"76803-970","phone":"+55 69 3219-7450","latitude":-8.71417,"longitude":-63.89832,"uct":-240,"website":""}

Dados para o ICAO SBRB:
{"id":6294,"iata":"RBR","icao":"SBRB","name":"Plácido de Castro International Airport","locat

Dados para o ICAO SBTG:
{"id":7410,"iata":"TJL","icao":"SBTG","name":"Plínio Alarcom Airport","location":"Três Lagoas, Mato Grosso do Sul, Brazil","street_number":"","street":"Avenida Antônio Trajano","city":"","county":"Três Lagoas","state":"Mato Grosso do Sul","country_iso":"BR","country":"Brazil","postal_code":"","phone":"","latitude":-20.753672,"longitude":-51.68211,"uct":-240,"website":""}

Dados para o ICAO SBTT:
{"id":7252,"iata":"TBT","icao":"SBTT","name":"Tabatinga International Airport","location":"Tabatinga, Amazonas, Brazil","street_number":"s/n","street":"Praça Marechal-do-ar Eduardo Gomes","city":"","county":"Tabatinga","state":"Amazonas","country_iso":"BR","country":"Brazil","postal_code":"69640-000","phone":"+55 97 3412-1100","latitude":-4.25038,"longitude":-69.93939,"uct":-240,"website":"http://www.infraero.gov.br/"}

Dados para o ICAO SBUA:
{"id":6812,"iata":"SJL","icao":"SBUA","name":"São Gabriel da Cachoeira Airport","location":"São Gabriel da Cachoeira, Amazonas, B

Dados para o ICAO SLVR:
{"id":8079,"iata":"VVI","icao":"SLVR","name":"Viru Viru International Airport","location":"Santa Cruz de la Sierra, Bolivia","street_number":"","street":"Avenida G77","city":"Santa Cruz de la Sierra","county":"","state":"Departamento de Santa Cruz","country_iso":"BO","country":"Bolivia","postal_code":"","phone":"+591 3 3385000","latitude":-17.64312,"longitude":-63.140755,"uct":-240,"website":"http://www.sabsa.aero/"}

Dados para o ICAO SMJP:
{"id":5794,"iata":"PBM","icao":"SMJP","name":"Johan Adolf Pengel International Airport","location":"Paramaribo, Suriname","street_number":"","street":"","city":"Paramaribo","county":"Zuid-Para","state":"Para","country_iso":"SR","country":"Suriname","postal_code":"","phone":"+597 401-348","latitude":5.456003,"longitude":-55.200058,"uct":-180,"website":"https://www.japi-airport.com/"}

Dados para o ICAO SNBR:
{"id":929,"iata":"BRA","icao":"SNBR","name":"Barreiras Airport","location":"Barreiras, Bahia, Brazil","street_number":"

Dados para o ICAO SWMW:
{"id":4537,"iata":"MBZ","icao":"SWMW","name":"Maués Airport","location":"Maués, Amazonas, Brazil","street_number":"1593","street":"Estrada São João","city":"","county":"Maués","state":"Amazonas","country_iso":"BR","country":"Brazil","postal_code":"69190-000","phone":"","latitude":-3.374394,"longitude":-57.723732,"uct":-240,"website":""}

Dados para o ICAO SWPI:
{"id":5922,"iata":"PIN","icao":"SWPI","name":"Júlio Belém Airport","location":"Parintins, Amazonas, Brazil","street_number":"1","street":"Estr. Odovaldo Novo","city":"","county":"Parintins","state":"Amazonas","country_iso":"BR","country":"Brazil","postal_code":"69152-470","phone":"+55 92 3533-2700","latitude":-2.669512,"longitude":-56.76967,"uct":-240,"website":""}

Dados para o ICAO SWTS:
{"id":7348,"iata":"TGQ","icao":"SWTS","name":"Tangará da Serra Airport","location":"Tangará da Serra, Mato Grosso, Brazil","street_number":"","street":"","city":"","county":"Tangará da Serra","state":"State of Mato Gros

## Tratamento dos dados
    A API acabou por não encontrar alguns ICAO dos aerodromos oque ocasiou linhas vazias na tabela, para melhor entrega dos dados tratei os removendo bem como 2 colunas sendo elas "error" e "message" pois evieram como NA.

In [86]:
# Breve Analise dos dados

# Verificar informações gerais sobre o DataFrame
print("\nInformações gerais sobre o DataFrame:")
print(df.info())

# Verificar valores ausentes em cada coluna
print("\nValores ausentes em cada coluna:")
print(df.isnull().sum())




Informações gerais sobre o DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             180 non-null    float64
 1   iata           180 non-null    object 
 2   icao           180 non-null    object 
 3   name           180 non-null    object 
 4   location       180 non-null    object 
 5   street_number  180 non-null    object 
 6   street         180 non-null    object 
 7   city           180 non-null    object 
 8   county         180 non-null    object 
 9   state          180 non-null    object 
 10  country_iso    180 non-null    object 
 11  country        180 non-null    object 
 12  postal_code    180 non-null    object 
 13  phone          180 non-null    object 
 14  latitude       180 non-null    float64
 15  longitude      180 non-null    float64
 16  uct            180 non-null    float64
 17  website        

In [88]:
# Tratamento de erros
df_aero = df.drop('error', axis=1)
df_aero.head()


,id,iata,icao,name,location,street_number,street,city,county,state,country_iso,country,postal_code,phone,latitude,longitude,uct,website
0,8847.0,YYZ,CYYZ,Toronto Pearson International Airport,"Toronto, Ontario, Canada",6301,Silver Dart Drive,Mississauga,,Ontario,CA,Canada,L5P 1B2,+1 416-247-7678,43.677720,-79.624820,-240.0,http://www.torontopearson.com/
1,53.0,ACC,DGAA,Kotoka International Airport,"Accra, Ghana",,Airport Road,Accra,Accra Metropolis,Greater Accra Region,GH,Ghana,,+233 30 277 6171,5.606095,-0.168124,0.0,http://www.gacl.com.gh/
2,3531.0,KAN,DNKN,Mallam Aminu Kano International Airport,"Kano, Nigeria",,Sani Abacha Way,Kano,,Kano,NG,Nigeria,,,12.045650,8.522076,60.0,http://www.faan.gov.ng/mallam-aminu-kano-inter...
3,4308.0,LOS,DNMM,Murtala Muhammed International Airport,"Lagos, Nigeria",,,Lagos,,Lagos,NG,Nigeria,,+234 1 493 3385,6.581819,3.321135,60.0,http://www.faan.gov.ng/mmia/
4,947.0,BRU,EBBR,Brussels Airport (Zaventem Airport),"Brussels, Belgium",,Leopoldlaan,Zaventem,Vlaams-Brabant,Vlaanderen,BE,Belgium,1930,+32 2 753 77 53,50.900997,4.485574,120.0,http://www.brusselsairport.be/


In [89]:
# Removendo as linhas null que nao foram encontradas nas requisições
colunas_a_verificar = ['id', 'iata', 'icao', 'name', 'location', 'street_number', 'street', 'city', 'county', 'state', 'country_iso', 'country', 'postal_code', 'phone', 'latitude', 'longitude', 'uct', 'website']

# Remover linhas com valores ausentes nas colunas especificadas
df_aero = df_aero.dropna(subset=colunas_a_verificar)


#Verificar informações gerais sobre o DataFrame
print("\nInformações gerais sobre o DataFrame:")
print(df_aero.info())

#Verificar valores ausentes em cada coluna
print("\nValores ausentes em cada coluna:")
print(df_aero.isnull().sum())

df_aero.head()


Informações gerais sobre o DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 180 entries, 0 to 199
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             180 non-null    float64
 1   iata           180 non-null    object 
 2   icao           180 non-null    object 
 3   name           180 non-null    object 
 4   location       180 non-null    object 
 5   street_number  180 non-null    object 
 6   street         180 non-null    object 
 7   city           180 non-null    object 
 8   county         180 non-null    object 
 9   state          180 non-null    object 
 10  country_iso    180 non-null    object 
 11  country        180 non-null    object 
 12  postal_code    180 non-null    object 
 13  phone          180 non-null    object 
 14  latitude       180 non-null    float64
 15  longitude      180 non-null    float64
 16  uct            180 non-null    float64
 17  website        180 n

,id,iata,icao,name,location,street_number,street,city,county,state,country_iso,country,postal_code,phone,latitude,longitude,uct,website
0,8847.0,YYZ,CYYZ,Toronto Pearson International Airport,"Toronto, Ontario, Canada",6301,Silver Dart Drive,Mississauga,,Ontario,CA,Canada,L5P 1B2,+1 416-247-7678,43.677720,-79.624820,-240.0,http://www.torontopearson.com/
1,53.0,ACC,DGAA,Kotoka International Airport,"Accra, Ghana",,Airport Road,Accra,Accra Metropolis,Greater Accra Region,GH,Ghana,,+233 30 277 6171,5.606095,-0.168124,0.0,http://www.gacl.com.gh/
2,3531.0,KAN,DNKN,Mallam Aminu Kano International Airport,"Kano, Nigeria",,Sani Abacha Way,Kano,,Kano,NG,Nigeria,,,12.045650,8.522076,60.0,http://www.faan.gov.ng/mallam-aminu-kano-inter...
3,4308.0,LOS,DNMM,Murtala Muhammed International Airport,"Lagos, Nigeria",,,Lagos,,Lagos,NG,Nigeria,,+234 1 493 3385,6.581819,3.321135,60.0,http://www.faan.gov.ng/mmia/
4,947.0,BRU,EBBR,Brussels Airport (Zaventem Airport),"Brussels, Belgium",,Leopoldlaan,Zaventem,Vlaams-Brabant,Vlaanderen,BE,Belgium,1930,+32 2 753 77 53,50.900997,4.485574,120.0,http://www.brusselsairport.be/


In [92]:
# Salvando o data
import os

# Especifique o caminho completo para a pasta onde deseja salvar o arquivo CSV
pasta_especifica = 'C:/Users/Didrich/Documents/Python Scripts/DataEnginnerTest/Aerodomos/Aerodomos_tratado'

# Certifique-se de que a pasta existe, caso contrário, crie-a
if not os.path.exists(pasta_especifica):
    os.makedirs(pasta_especifica)

# Especifique o nome do arquivo CSV
nome_arquivo_csv = 'df_aero.csv'

# Crie o caminho completo para o arquivo CSV
caminho_arquivo_csv = os.path.join(pasta_especifica, nome_arquivo_csv)

# Exporte o DataFrame para o arquivo CSV
df_aero.to_csv(caminho_arquivo_csv, index=False, sep=';')  # sep=';' para usar ponto e vírgula como separador

print(f'O DataFrame foi exportado para {caminho_arquivo_csv}')

O DataFrame foi exportado para C:/Users/Didrich/Documents/Python Scripts/DataEnginnerTest/Aerodomos/Aerodomos_tratado\df_aero.csv
